### Instalações

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
plt.style.use("ggplot")

import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from keras.models import Sequential
from keras import Model, Input
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Dropout, Conv1D, MaxPooling1D, Embedding, TimeDistributed, Bidirectional,GlobalMaxPooling1D

#!pip install -U gensim
from gensim.models import Word2Vec

#!pip install livelossplot
from livelossplot.tf_keras import PlotLossesCallback

#!pip install seqeval
from seqeval.metrics import f1_score, classification_report, precision_score, recall_score, accuracy_score
from seqeval.scheme import IOB2
#! pip install plot_keras_history
from plot_keras_history import plot_history
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

[nltk_data] Downloading package punkt to /Users/lucelia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-07-19 15:10:15.790848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Dados

In [3]:
import pandas as pd

df = pd.read_csv('./CSVs/V2/DODFCorpus_contratos_licitacoes_v2.csv', dtype=str)
df = df.drop(['Unnamed: 0','Unnamed: 0.1'], axis =1)

In [4]:
##Insere um espaço entre as entidade e insere um espaco apos :  : -> Processo:0...
import re
def correct_space_before_numeric_entities(string):
    result = re.sub(r'([A-Za-z]:)[0-9]', r'\1 ', string)
    result = result.replace("\n", " ")
    return result

In [5]:
df['texto']= df['texto'].map(correct_space_before_numeric_entities)

In [6]:
df.tipo_rel.unique()

array(['REL_AVISO_LICITACAO', 'REL_SUSPENSAO_LICITACAO',
       'REL_EXTRATO_CONTRATO', 'REL_ADITAMENTO_CONTRATO',
       'REL_ANUL_REVOG_LICITACAO', 'REL_EXTRATO_CONVENIO'], dtype=object)

In [7]:
#Concatena o id_ato com o id_dodf
df['id_ato'] = df['id_dodf'] + '-' + df['id_rel']
#data

In [8]:
#Lista o tipo de relatório
result =df.groupby('tipo_rel')['id_ato'].nunique()
print(result.sort_values(ascending=False))

tipo_rel
REL_EXTRATO_CONTRATO        1734
REL_ADITAMENTO_CONTRATO     1551
REL_AVISO_LICITACAO          639
REL_SUSPENSAO_LICITACAO       82
REL_ANUL_REVOG_LICITACAO      52
REL_EXTRATO_CONVENIO          32
Name: id_ato, dtype: int64


In [13]:
tipo_ato = df.query("tipo_rel == 'REL_EXTRATO_CONTRATO'").reset_index(drop=True)

In [14]:
#Lista as entidades do tipo de ato
result = tipo_ato.groupby('tipo_ent')['id_ato'].nunique()
print(result)
#print(result.sort_values(ascending=False))

tipo_ent
EXTRATO_CONTRATO            1734
cnpj_entidade_contratada     479
cnpj_orgao_contratante       135
codigo_siggo                 175
data_assinatura_contrato    1286
entidade_contratada         1721
fonte_recurso               1317
natureza_despesa            1068
nome_responsavel             185
nota_empenho                1214
numero_contrato             1714
objeto_contrato             1724
orgao_contratante           1700
processo_gdf                1721
programa_trabalho           1280
unidade_orcamentaria        1045
valor_contrato              1566
vigencia_contrato           1664
Name: id_ato, dtype: int64


In [15]:
#cria um novo dataset apenas com o tipo de entidade EXTATO_CONTRATO (TEXTO)
#tipo_ato1 = tipo_ato.query("tipo_ent == 'AVISO_ANUL_REV_LICITACAO'").reset_index(drop=True)
#tipo_ato1 = tipo_ato.query("tipo_ent == 'EXTRATO_CONTRATO'").reset_index(drop=True)
#tipo_ato1 = tipo_ato.query("tipo_ent == 'EXTRATO_CONTRATO'").reset_index(drop=True)
#tipo_ato1 = tipo_ato.query("tipo_ent == 'EXTRATO_ADITAMENTO_CONTRATUAL'").reset_index(drop=True)
#tipo_ato1 = tipo_ato.query("tipo_ent == 'EXTRATO_CONVENIO'").reset_index(drop=True)
tipo_ato1 = tipo_ato.query("tipo_ent == 'EXTRATO_CONTRATO'").reset_index(drop=True)
#tipo_ato1 = tipo_ato.query("tipo_ent == 'EXTRATO_ADITAMENTO_CONTRATUAL'").reset_index(drop=True)
#tipo_ato1

In [16]:
#criando novo dataframe somente com os textos
data = tipo_ato1['texto'].reset_index(drop=True)
#data.to_csv("./CSVs/V2/REL_ANUL_REVOG_LICITACAO.csv")
data

0       EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...
1       EXTRATO DO CONTRATO PARA AQUISICAO DE BENS No ...
2       EXTRATO DO CONTRATO No 32/2018 AQUISICAO DE BE...
3       EXTRATO DE CONTRATO CONTRATO No 9020. ASSINATU...
4       EXTRATO DO CONTRATO No 01/2018, NOS TERMOS DO ...
                              ...                        
1729    EXTRATO DO CONTRATO PARA AQUISIÇÃO DE BENS Nº ...
1730    EXTRATO DO CONTRATO DE EXECUÇÃO DE OBRAS PARA ...
1731    EXTRATO DO CONTRATO Nº 45.926/2022 Processo: 0...
1732    EXTRATO DO CONTRATO Nº 45832/2022 Processo: 00...
1733    EXTRATO DO CONTRATO Nº 45927/2022 Processo: 00...
Name: texto, Length: 1734, dtype: object

In [17]:
# CODIGO RESPONSAVEL PELA LIMPEZA DO TEXTO DA BASE OURO
import re

df = []

for i in data:
    i = i.replace("\n", " ")
    i = re.sub('xx[a-z]{1,10}', '', i)
    aux = ' '.join(i.split())
    df.append(aux)
    

In [14]:
#df

## Aplicando a Supervisão Fraca

In [18]:
# CODIGO DE APLICACAO DA SUPERVISAO FRACA

#Para rodar aditamento de contrato 
#import dodfSkweak_Aditamento_2
#skw = dodfSkweak_Aditamento_2.SkweakAditamento(df)

#Para rodar Aviso Licitação
#import dodfSkweak_Aviso_Licitacao
#skw = dodfSkweak_Aviso_Licitacao.SkweakAvisoLicitacao(df)

#Para rodar Suspensao 
#import dodfSkweak_Suspensao_1
#skw = dodfSkweak_Suspensao_1.SkweakSuspensao(df)

#Para rodar Anulacao e Revogação
#import dodfSkweak_Anul_Revog_1
#skw = dodfSkweak_Anul_Revog_1.SkweakAnulRevog(df)

#Para rodar extrato de contrato
import dodfSkweak_Contrato
skw = dodfSkweak_Contrato.SkweakContratos(df)

#Para rodar Convenio
#import dodfSkweak_Convenio
#skw = dodfSkweak_Convenio.SkweakConvenio(df)

skw.apply_label_functions()
skw.train_HMM_Dodf()

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'lemmatizer'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


Starting iteration 1
Number of processed documents: 1000
Finished E-step with 1734 documents
Starting iteration 2


         1      -90438.4914             +nan


Number of processed documents: 1000
Finished E-step with 1734 documents
Starting iteration 3


         2      -88696.5614       +1741.9300


Number of processed documents: 1000
Finished E-step with 1734 documents
Starting iteration 4


         3      -88603.0361         +93.5253


Number of processed documents: 1000
Finished E-step with 1734 documents


         4      -88579.9800         +23.0561


Write to ./data/reuters_small.spacy...done


In [ ]:
#List os tokens
#skw.list_spans_all()
skw.list_spans_specific(1)

In [ ]:
#Imprimi o Dataset gerado pela supervisão 
data_label=skw.get_hmm_dataframe()
data_label
# converting to CSV file
#data_label.to_csv("./CSVs/V2/supervisao_fraca/CONVENIO.csv")

In [19]:
#Aplica o IOB com spacy tokenizer
corpus = skw.get_IOB()
#corpus

In [20]:
#Separa Tokens e os labels
termos_skweak = []
debug=set()
labels_skweak = []
for ato in corpus:
    x = []
    y = []
    for termo in ato:
        termo_label = termo.split()
        x.append(termo_label[0])
        y.append(termo_label[1])
        classes=debug.add(termo_label[1])
    termos_skweak.append(x)
    labels_skweak.append(y) 

In [21]:
debug

{'B-data_assinatura_contrato',
 'B-natureza_despesa',
 'B-nota_empenho',
 'B-numero_contrato',
 'B-processo_gdf',
 'B-programa_trabalho',
 'B-unidade_orcamentaria',
 'B-valor_contrato',
 'I-data_assinatura_contrato',
 'I-natureza_despesa',
 'I-nota_empenho',
 'I-numero_contrato',
 'I-processo_gdf',
 'I-programa_trabalho',
 'I-unidade_orcamentaria',
 'O'}

In [22]:
max_length = 400

### Ajustes das labels e dicionários

In [23]:
def remove_wrong_tags(label_list):
  for label in label_list:
    for idx,w in enumerate(label):
      if w in ['B-11','B-12','B-50', 'B-60', 'I-2']:
        label[idx] = 'O'

In [24]:
remove_wrong_tags(labels_skweak)

### Transformação dos dados de Treino

In [25]:
words = set()

for act in termos_skweak:
    for word in act:
        words.add(word)
#convertendo o set em uma lista
words = list(words)

words.append("ENDPAD")
words.append("UNK")

words_amt = len(words)

In [26]:
tags = set()

for label in labels_skweak:
    for tag in label:
        tags.add(tag)
tags = list(tags)
tags_amt = len(tags)

In [27]:
lab_enc = LabelEncoder()

lab_enc.fit(words)
words_i = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))

i_words = {}

for key in words_i:
  i_words[words_i[key]] = key

In [28]:
lab_enc = LabelEncoder()

lab_enc.fit(tags)
tags_i = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))

i_tags = {}

for key in tags_i:
  i_tags[tags_i[key]] = key

In [29]:
def transform_data(x,y):
    X,Y = [],[]

    for act in x:
        aux = []
        for word in act:
            aux.append(words_i[word])
        X.append(aux)

    for label in y:
        aux = []
        for word in label:
            aux.append(tags_i[word])
        Y.append(aux)

    return X,Y

In [30]:
inputs, targets = transform_data(termos_skweak,labels_skweak)

In [31]:
inputs = pad_sequences(maxlen=max_length, sequences=inputs, padding="post", value=words_i['ENDPAD'])
targets = pad_sequences(maxlen=max_length, sequences=targets, padding="post", value=tags_i["O"])

### Funções auxiliares

In [32]:
def convert_values(index_array,y_test):
  pred_tags = []
  real_tags = []

  for act in index_array:
    act_tags = []
    for w in act:
      act_tags.append(i_tags[w])
    pred_tags.append(act_tags)

  for ato in y_test:
    tags_ato = []
    for palavra in ato:
      tags_ato.append(i_tags[palavra])
    real_tags.append(tags_ato)

  return real_tags, pred_tags

# K-fold

In [33]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state = 42)

### CNN-BiLSTM

In [36]:

#Extrato de Contrato
cnnbilstm_lr = 0.003
cnnbilstm_units = 340
cnnbatch_size = 12
cnnepochs = 50

'''
#Extrato de Convenio
cnnbilstm_lr = 0.003
cnnbilstm_units = 340
cnnbatch_size = 12
cnnepochs = 50


#Aviso de Licitação
cnnbilstm_lr = 0.0055
cnnbilstm_units = 140
cnnbatch_size = 5
cnnepochs = 50


#Suspensão
cnnbilstm_lr = 0.0065
cnnbilstm_units = 200
cnnbatch_size = 5
cnnepochs = 50 



#Extrato de Contrato
cnnbilstm_lr = 0.003
cnnbilstm_units = 340
cnnbatch_size = 12
cnnepochs = 50


#Aditamento Contratual
cnnbilstm_lr = 0.001
cnnbilstm_units = 180
cnnbatch_size = 12
cnnepochs = 50




#Extrato de Convenio
cnnbilstm_lr = 0.003
cnnbilstm_units = 340
cnnbatch_size = 12
cnnepochs = 20

#Extrato de Contrato
cnnbilstm_lr = 0.003
cnnbilstm_units = 340
cnnbatch_size = 12
cnnepochs = 20


#Revogação
cnnbilstm_lr = 0.008
cnnbilstm_units = 105
cnnbatch_size = 5
cnnepochs = 50
'''

'\n#Extrato de Convenio\ncnnbilstm_lr = 0.003\ncnnbilstm_units = 340\ncnnbatch_size = 12\ncnnepochs = 50\n\n\n#Aviso de Licitação\ncnnbilstm_lr = 0.0055\ncnnbilstm_units = 140\ncnnbatch_size = 5\ncnnepochs = 50\n\n\n#Suspensão\ncnnbilstm_lr = 0.0065\ncnnbilstm_units = 200\ncnnbatch_size = 5\ncnnepochs = 50 \n\n\n\n#Extrato de Contrato\ncnnbilstm_lr = 0.003\ncnnbilstm_units = 340\ncnnbatch_size = 12\ncnnepochs = 50\n\n\n#Aditamento Contratual\ncnnbilstm_lr = 0.001\ncnnbilstm_units = 180\ncnnbatch_size = 12\ncnnepochs = 50\n\n\n\n\n#Extrato de Convenio\ncnnbilstm_lr = 0.003\ncnnbilstm_units = 340\ncnnbatch_size = 12\ncnnepochs = 20\n\n#Extrato de Contrato\ncnnbilstm_lr = 0.003\ncnnbilstm_units = 340\ncnnbatch_size = 12\ncnnepochs = 20\n\n\n#Revogação\ncnnbilstm_lr = 0.008\ncnnbilstm_units = 105\ncnnbatch_size = 5\ncnnepochs = 50\n'

In [34]:
act_name='EXTRATO_CONTRATO'
act_name

'EXTRATO_CONTRATO'

In [37]:
#import os, pickle
from tensorflow.keras.layers import BatchNormalization, Dropout, Conv1D, MaxPooling1D, Embedding, TimeDistributed, Bidirectional,GlobalMaxPooling1D

acc_3 = []
loss_3 = []
f1_3 = []
reports_3 = []
fold = 0

for train, test in kfold.split(inputs, targets):
 
  x_train = inputs[train]
  x_test = inputs[test]
  y_train = targets[train]
  y_test = targets[test]
  
  #x_test = test_splits[act_name][fold]['x']
  #y_test = test_splits[act_name][fold]['y']
          
  cnn_bilstm = Sequential()
  cnn_bilstm.add(Embedding(input_dim=words_amt, output_dim=50, input_length=max_length))
  cnn_bilstm.add(Conv1D(filters=tags_amt, kernel_size=3, padding='same', activation='relu'))
  cnn_bilstm.add(BatchNormalization())
  cnn_bilstm.add(Dropout(0.5))
  cnn_bilstm.add(Bidirectional(LSTM(cnnbilstm_units, return_sequences=True)))
  cnn_bilstm.add(TimeDistributed(Dense(tags_amt, activation="softmax")))

  adam = Adam(learning_rate=cnnbilstm_lr)

  cnn_bilstm.compile(optimizer=adam,loss="sparse_categorical_crossentropy",metrics=["accuracy"])

  early_stopping = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
  callbacks = [early_stopping]

  history = cnn_bilstm.fit(x_train,y_train,batch_size=cnnbatch_size, epochs=cnnepochs)

  scores = cnn_bilstm.evaluate(x_test,y_test,verbose=0)
  acc_3.append(scores[1])
  loss_3.append(scores[0])

  predictions = cnn_bilstm.predict(x_test, verbose=0)
  #y_pred = cnn_bilstm.predict(x_test, verbose=0)
 
  predictions = np.argmax(predictions, axis=-1)
  y_pred = np.argmax(predictions, axis=-1)
  real_tags, pred_tags = convert_values(predictions,y_test)
  

  
  f1_3.append(f1_score(real_tags, pred_tags))
  reports_3.append(classification_report(real_tags, pred_tags))
  
  r = classification_report(real_tags,pred_tags, output_dict=True, mode='strict', scheme=IOB2)
  
  name =  act_name + '_f' + str(fold) +".npy"
  np.save('./Results/V2/CISTI/27_06/WS_BILSTM/'+name, r)

  fold = fold + 1
  reports_3.append(r)

Epoch 1/50
116/116 [==============================] - 912s 8s/step - loss: 0.1633 - accuracy: 0.9672
Epoch 2/50
116/116 [==============================] - 442s 4s/step - loss: 0.0457 - accuracy: 0.9881
Epoch 3/50
116/116 [==============================] - 443s 4s/step - loss: 0.0181 - accuracy: 0.9954
Epoch 4/50
116/116 [==============================] - 452s 4s/step - loss: 0.0098 - accuracy: 0.9975
Epoch 5/50
116/116 [==============================] - 462s 4s/step - loss: 0.0060 - accuracy: 0.9984
Epoch 6/50
116/116 [==============================] - 488s 4s/step - loss: 0.0042 - accuracy: 0.9988
Epoch 7/50
116/116 [==============================] - 499s 4s/step - loss: 0.0029 - accuracy: 0.9992
Epoch 8/50
116/116 [==============================] - 488s 4s/step - loss: 0.0024 - accuracy: 0.9993
Epoch 9/50
116/116 [==============================] - 332s 3s/step - loss: 0.0022 - accuracy: 0.9994
Epoch 10/50
116/116 [==============================] - 303s 3s/step - loss: 0.0019 - accura

In [47]:
#pred_tags


In [48]:
f1_3[0]

0.0

In [41]:
np.mean(f1_3)

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [50]:
np.std(f1_3)

0.0

In [34]:
act_name

'ANUL_REV_LICITACAO'

In [99]:
print(reports_3[0])

                  precision    recall  f1-score   support

numero_licitacao       1.00      0.54      0.70        13

       micro avg       1.00      0.54      0.70        13
       macro avg       1.00      0.54      0.70        13
    weighted avg       1.00      0.54      0.70        13

